In [1]:
###### Done

# resolver casos donde el dm se pasa de los 300 caracteres
#  > analizar si nos vamos a pasar de los 300 char antes de clickear Connect

# implementar proceso contínuo

# guardar data en gsheets > incluyendo casos donde nos pasaríamos de los 300 char u otro error para ir solucionando y mejorando

# arreglado: la dm_position iba perdiendo characteres lol

# Asegurarnos de que el título amerite mandarle un DM > evitar casos como title = "Searching"...

# Guardar registro de todos los perfiles a los que le enviamos connect + dm

# Guardar registro de todas las corridas en "Runs" tab

# Armar proceso para DMs sin Connect

# nunca mandar 2 veces un dm

# Llevar registro de si ya le mandamos DM (id = link)

###### Building

# mantener registro con 1 usuario por fila (id = link) e ir actualizando la data

# evitar filas duplicadas en sheets wo Connect



###### Next steps

# Enviar mensaje a lucas > no veo que tenga... Preguntarle si usa LinkedIn jaja

# Scrapear start ups y sus empleados para mandarles DM a 1 por empresa



# 7 días entre distintos mensajes



###### Not Prioritized






# Sheets: https://docs.google.com/spreadsheets/d/1DTb87DtRRzZwFoPz-odPbCcNmEKGWKsUwYWQRpON68c/edit#gid=1167036033



In [99]:
##########################################################################
#This script is built for charging activation fee monthly
#Created by Mengying Liu mengying.liu@uber.com
#If there is any error you met running this script, Please contact Mengying Liu
###########################################################################

##############################LOAD LIBRARY START###############################
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime
import traceback, os
from datetime import timedelta
import pandas as pd
import xlrd
import time
import sys
import pygsheets as py
import numpy as np

from IPython.display import clear_output

import subprocess

from bs4 import BeautifulSoup

# untraceable chromedriver
import undetected_chromedriver.v2 as uc


##############################LOAD LIBRARY END#################################

# Inputs

In [3]:
## Google Sheets ID 

#sheet_id = '14dtIAqerziwN9xldTXyZATgEkrf4HZJxH455HYoyC4w'

sheet_id = '1DTb87DtRRzZwFoPz-odPbCcNmEKGWKsUwYWQRpON68c'

## File locations

#chromedriver_file_loc = "/Users/matiastrebinonixon/Documents/Auto Flows/chromedriver" > de ahora en más usamos el servicio de undetectable_chrome

#google_sheets_client_secret_file_loc = '/Users/matias/Documents/Python/client_secret_61294989490-topfi2gobmjntvuvkl3ntoafd801m11v.apps.googleusercontent.com.json'
#google_sheets_client_secret_file_loc = '/Users/matias/Documents/Python/client_secret_61294989490-ael82691t87n8mokcn5u2c647c6v8nif.apps.googleusercontent.com.json'

# usamos el acceso de matias.trebino@homie.mx porque es el unico que me funciona actualmente
google_sheets_client_secret_file_loc = '/Users/matias/Documents/Python/client_secret_272462785567-kj1ont1cp6hea636sfsu0l59r2bjmcon.apps.googleusercontent.com.json'

gs = py.authorize(client_secret = google_sheets_client_secret_file_loc)



# Functions

In [4]:
def undetectable_options():
    
    global options
    global user_agent_n
    
    #options = webdriver.ChromeOptions() # normal
    options = uc.ChromeOptions() # undetectable
    
    options.add_argument("--start-maximized")
    #options.add_experimental_option("excludeSwitches", ["enable-automation"]) # > no me funciona en undetectable chromedriver
    #options.add_experimental_option('useAutomationExtension', False) # > no me funciona en undetectable chromedriver
    options.add_argument('--disable-notifications')
    options.add_argument("--mute-audio")
    options.add_argument("--incognito") ## chrome incognito mode
    #options.add_argument("--disable-web-security"); creo que viejo y no funciona, no usar, lo puse tratando de que no se eviten los pop-ups, eso lo logré con el siguiente
    options.add_argument("--disable-popup-blocking");


    # Options to avoid bot detection
    #options.add_argument('--disable-blink-features=AutomationControlled') # el undetectable chromedriver recomienda sacarlo
    options.add_argument("--window-size=1282,814")
    #options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36")
    
    ####### rotate user agent
    ######
    ######if user_agent_n%3 == 0: options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko")
    ######if user_agent_n%3 == 1: options.add_argument("--user-agent=Chrome/98.0.4758.109 ")
    ######if user_agent_n%3 == 2: options.add_argument("--user-agent=Safari/537.36")
    ######
    ######user_agent_n += 1
    
    # non-rotating user-agent
    
    #options.add_argument("--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.109 Safari/537.36")
    
    #add rotating ip - first in table

    #rotating_ip = get_latam_proxy()
    #options.add_argument('--proxy-server='+rotating_ip)
    
    # add free vpn chrome extension > no me funca
    
    #options.add_extension('/Users/matias/Documents/Python/free_vpn_chrome_extension.crx')

In [5]:
# Google Sheets aux functions

def sheet_to_df(sheet_array, index_col_array = []):
    
    aux_df = pd.DataFrame(sheet_array)
    aux_df = aux_df.replace('',np.nan)
    aux_df = aux_df.dropna(how = "all", axis = 0)
    aux_df = aux_df.dropna(how = "all", axis = 1)
    aux_df.columns = aux_df.loc[0]
    aux_df = aux_df.drop(index = [0], axis = 0).reset_index(drop = True)
    
    if len(index_col_array) > 0:
        
        aux_keys = []
        
        for i in index_col_array:
            aux_keys.append(aux_df.columns[i])
                
        aux_df = aux_df.set_index(keys = aux_keys, drop = True)
    
    aux_df = aux_df.replace(np.nan,'')
    
    return aux_df 

In [6]:
def scroll():
    
    driver.execute_script("window.scrollTo(0,9999999);")
    

In [7]:
def copy(data):
    
    subprocess.run("pbcopy", universal_newlines=True, input=data)
    

In [23]:
def get_gsheets_inputs():
    
    global dms_tab
    global dms_df
    
    global runs_tab
    global runs_df
    
    global runs_wo_connect_tab
    global runs_wo_connect_df
    
    global dms_wo_connect_tab
    global dms_wo_connect_df
    
    global searches_df
    
    global company_exclusions_list
   
    global dm_message
    global dm_message_direct_1_full_title
    global dm_message_direct_1_full_body
    
    global start_ups_searches_tab
    global start_ups_searches_df
    global start_ups_companies_tab
    global start_ups_companies_df
    global start_ups_people_tab
    global start_ups_people_df
    global start_ups_dms_tab
    global start_ups_dms_df
    
    dms_tab = gs.open_by_key(sheet_id).worksheet_by_title('DMs')
    
    dms_df = sheet_to_df(dms_tab.get_all_values())
    
    
    runs_tab = gs.open_by_key(sheet_id).worksheet_by_title('Runs')
    
    runs_df = sheet_to_df(runs_tab.get_all_values())
    
    
    dms_wo_connect_tab = gs.open_by_key(sheet_id).worksheet_by_title('DMs w/o Connect')
    
    dms_wo_connect_df = sheet_to_df(dms_wo_connect_tab.get_all_values())
    
    
    runs_wo_connect_tab = gs.open_by_key(sheet_id).worksheet_by_title('Runs w/o Connect')
    
    runs_wo_connect_df = sheet_to_df(runs_wo_connect_tab.get_all_values())
    
    
    searches_tab = gs.open_by_key(sheet_id).worksheet_by_title('Searches')
    
    searches_df = sheet_to_df(searches_tab.get_all_values())
    
    
    company_exclusions_tab = gs.open_by_key(sheet_id).worksheet_by_title('Company Exclusions')
    
    company_exclusions_df = sheet_to_df(company_exclusions_tab.get_all_values())
    
    company_exclusions_list = company_exclusions_df['Company Exclusions'].tolist()
    
    
    dm_message_tab = gs.open_by_key(sheet_id).worksheet_by_title('Message')
    
    dm_message = dm_message_tab.cell('A1').value.lstrip('\n').rstrip('\n')
    
    
    dm_message_direct_1_full_tab = gs.open_by_key(sheet_id).worksheet_by_title('DM')
    
    dm_message_direct_1_full_title = dm_message_direct_1_full_tab.cell('A1').value.lstrip('\n').rstrip('\n')
    
    dm_message_direct_1_full_body = dm_message_direct_1_full_tab.cell('A2').value.lstrip('\n').rstrip('\n')
    
    
    start_ups_searches_tab = gs.open_by_key(sheet_id).worksheet_by_title('StartUps Searches')
    
    start_ups_searches_df = sheet_to_df(start_ups_searches_tab.get_all_values())
    
    
    start_ups_companies_tab = gs.open_by_key(sheet_id).worksheet_by_title('StartUps Companies')
    
    start_ups_companies_df = sheet_to_df(start_ups_companies_tab.get_all_values())
    
    
    start_ups_people_tab = gs.open_by_key(sheet_id).worksheet_by_title('StartUps People')
    
    start_ups_people_df = sheet_to_df(start_ups_people_tab.get_all_values())
    
    
    start_ups_dms_tab = gs.open_by_key(sheet_id).worksheet_by_title('StartUps DMs')
    
    start_ups_dms_df = sheet_to_df(start_ups_dms_tab.get_all_values())
    
    

In [9]:
def log_in_to_linkedin():
    
    driver.get('https://www.linkedin.com/')
    
    ## insert if we want to automate, it is now easier for us to do it

In [10]:
#run_number_list = runs_df['Run'].tolist()

def get_run_number(runs_with_or_wo_connect_df):

    a = 0
    last_run_number = 0

    while a < len(runs_with_or_wo_connect_df):

        try:

            run_number_a = int(runs_with_or_wo_connect_df['Run'][a])

            if run_number_a > last_run_number: last_run_number = run_number_a

        except: 
            
            pass
            
        a += 1

    run_number = last_run_number + 1

    return run_number

def now_str():
    
    return datetime.now().strftime("%Y-%m-%d %H:%M")


In [11]:
def get_page_number():
    
    scroll() # gotta scroll so pagination loads
    time.sleep(1)
    
    get_try = 0
    
    while True:
        
        soup = BeautifulSoup(driver.page_source,'html.parser')
        
        try:
            page_number = int(soup.find('ul',{'class':'artdeco-pagination__pages'}).find('button',{'aria-current':'true'}).get_text().replace('Page ',''))
            
            if page_number > 0: 
                
                return page_number
            
            else:
                
                get_try =+ 1
                print('New try to get new page number')
                time.sleep(1)
        
        except:
            get_try =+ 1
            print('New try to get new page number')
            time.sleep(1)
            
        if get_try == 30:
            print('Failed to get new page number')
            print('Process shutdown')
            sys.exit()
            return 'Error'
            

In [12]:
def click_connect_button():

    connect_try = 0

    while connect_try < 6:
           
        try:
            button = driver.find_element(By.XPATH,'//*[@aria-label="Invite '+name+' to connect"]')
            driver.execute_script("window.scrollTo(0,"+str(button.location['y']-200)+");") # scroll into view
            button.click()
            time.sleep(1)
            print('Clicekd Connect button succesfully')
            return('success')

        except:

            try:
                # connect button
                button = driver.find_element(By.XPATH,'/html/body/div[5]/div[3]/div[2]/div/div[1]/main/div/div/div[1]/ul/li['+str(j+1)+']/div/div/div[3]/div/button')
                driver.execute_script("window.scrollTo(0,"+str(button.location['y']-200)+");") # scroll into view
                button.click()
                time.sleep(1)
                print('Clicekd Connect button succesfully')
                return('success')

            except:
                connect_try += 1
                print('Try {} to click Connect button unsusccesfull'.format(connect_try))
                time.sleep(1)

    return 'failed'


In [151]:
def undetectable_options():
    
    global options
    global user_agent_n
    
    options = webdriver.ChromeOptions() # normal
    #options = uc.ChromeOptions() # undetectable
    
    options.add_argument("--start-maximized")
    options.add_experimental_option("excludeSwitches", ["enable-automation"]) # > no me funciona en undetectable chromedriver
    options.add_experimental_option('useAutomationExtension', False) # > no me funciona en undetectable chromedriver
    options.add_argument('--disable-notifications')
    options.add_argument("--mute-audio")
    options.add_argument("--incognito") ## chrome incognito mode
    #options.add_argument("--disable-web-security"); creo que viejo y no funciona, no usar, lo puse tratando de que no se eviten los pop-ups, eso lo logré con el siguiente
    options.add_argument("--disable-popup-blocking");


    # Options to avoid bot detection
    options.add_argument('--disable-blink-features=AutomationControlled') # el undetectable chromedriver recomienda sacarlo
    options.add_argument("--window-size=1282,814")
    #options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36")
    
    ####### rotate user agent
    ######
    ######if user_agent_n%3 == 0: options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko")
    ######if user_agent_n%3 == 1: options.add_argument("--user-agent=Chrome/98.0.4758.109 ")
    ######if user_agent_n%3 == 2: options.add_argument("--user-agent=Safari/537.36")
    ######
    ######user_agent_n += 1
    
    # non-rotating user-agent
    
    options.add_argument("--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36")
    
    #add rotating ip - first in table

    #rotating_ip = get_latam_proxy()
    #options.add_argument('--proxy-server='+rotating_ip)
    
    # add free vpn chrome extension > no me funca
    
    #options.add_extension('/Users/matias/Documents/Python/free_vpn_chrome_extension.crx')

    


In [236]:
def get_linkedin_results_page(get_page_max=True):

    global pagination
    global page_current
    global page_max
    
    pagination = None
    
    pagination_try = 0
    
    while pagination == None:

        scroll()

        pagination_try += 1
        
        time.sleep(1)

        try:

            soup = BeautifulSoup(driver.page_source,'html.parser')

            pagination = soup.find("ul", {"class": "artdeco-pagination__pages"})

            page_current = int(pagination.find("button", {"aria-current": "true"}).get_text().replace('\n',''))
            
            pages_html = pagination.find_all("li", {"class": "artdeco-pagination__indicator"})

            if get_page_max: page_max = int(pages_html[len(pages_html)-1].get_text().replace('\n','').lstrip(' ').rstrip(' '))

        except:

            if pagination_try > 20: 
                
                print('Failed to get results pagination')
                sys.exit()
                

In [237]:
def scrape_people():
    
    global people_results_df
    global page_current
    global page_max
    global results
    
    people_results_df = pd.DataFrame(columns = ['name','position','company','link','action','timestamp'])
    
    # Get pagination
    
    get_linkedin_results_page(get_page_max=True)
    
    # Loop through results pages
        
    while True:
        
        # para cada página de resultados
        
        
        # Print progress
        
        clear_output(wait=True)
        print('Page {} of {}'.format(page_current,page_max))
        print('{}%'.format(round((page_current-1)*100.0/page_max,1)))
        
        # loop to check we got 10 results (there are always 10 results, maybe unless you are in the final results page)

        results = []
        
        results_check_try = 0

        while True:

            results_check_try += 1
            
            scroll()

            time.sleep(1)

            soup = BeautifulSoup(driver.page_source,'html.parser')

            results = soup.find_all("li", {"class": "reusable-search__result-container"})
            
            if results_check_try > 10:
                
                print('Error: cannot check that results are correct')
                sys.exit()
                
            if   len(results) >= 8: break
            elif (len(results) > 0) & (page_current == page_max): break
            
        # Iterate though all people
        
        for j in range(len(results)):
        
            ## get details of every results
            
            
            # name

            name = results[j].find('span',{'class':'entity-result__title-text'}).find("a", {"class": "app-aware-link"}).get_text().replace('\n','')

            if 'View' in name: name = name[:name.find('View')]
            
            print('Name: {}'.format(name))

            
            # position

            try: 
                position_full = results[j].find('div',{'class':'entity-result__primary-subtitle'}).get_text().replace('\n','')
            except:
                position_full = ''
            
            if   ' at ' in position_full: position = position_full[:position_full.find(' at ')]
            elif ' @ '  in position_full: position = position_full[:position_full.find(' @ ')]
            else: position = position_full
            
            print('Position: {}'.format(position))

            
            # link

            if name == 'LinkedIn Member': link = ''

            else:

                link_full = results[j].find('span',{'class':'entity-result__title-text'}).find("a", {"class": "app-aware-link"})['href']
                
                link = link_full[:link_full.find('?')]

                
            # company

            try: 
                sub_position_full = results[j].find('p',{'class':'entity-result__summary'}).get_text().replace('\n','')
            except:
                sub_position_full = 'empty'

            if 'Current: ' not in sub_position_full: sub_position = ''

            else: sub_position = sub_position_full[9:]

            if   ' at ' in position_full: company = position_full[position_full.find(' at ')+4:]
            elif ' @ '  in position_full: company = position_full[position_full.find(' @ ')+4:]
            
            elif ' at ' in sub_position: company = sub_position[sub_position.find(' at ')+4:]
            elif ' @ '  in sub_position: company = sub_position[sub_position.find(' @ ')+4:]

            else: company = ''
            
            print('Company: {}'.format(company))

            
            # action available
            
            try: 
                #action = results[j].find('button')['aria-label']
                action = results[j].find("div", {"class": "entity-result__actions"}).find("span", {"class": "artdeco-button__text"}).get_text().lstrip('\n').rstrip('\n').lstrip(' ').rstrip(' ')
            except:
                action = 'None'
                
            print('Action available: {}'.format(action))
            
            
            
            # Save data onto df
            
            people_results_df.loc[len(people_results_df)] = [name,position,company,link,action,now_str()]
            
            print('Saved data onto df')
        

        # click next page of results
        
        scroll()
        time.sleep(1)
        print('')
        next_try = 0
        

        
        #if page_current == 3: 
        #    print('intentional stop after 3 pages to check results')
        #    break 
            
            
            
        # Open next results page
        
        if page_current == page_max: 
            print('Finished')
            break
        
        next_try = 0
        
        while next_try < 10:
            
            failed_next_try = 0
            
            try:
                #driver.find_element(By.XPATH,"//*[contains(text(), 'Next')]")#.click()
                driver.find_element(By.XPATH,'//*[@aria-label="Next"]').click()
                time.sleep(6)
                print('Clicked next results page')
                next_try += 1

            except:
                print('Failed to click next resulst page')
                time.sleep(1)
                failed_next_try = 1
                continue
        
            try:
                # click No Results found button
                driver.find_element(By.XPATH,'//*[@data-test="no-results-cta"]').click()
                time.sleep(5)
                print('Detected results LinkedIn error: Clicked re-load results button.')

            except:
                pass
            
            if failed_next_try == 1: continue
            
            
            # check if new page is ok
            
            page_current_check = page_current
            
            #new_page_n = get_page_number()
            get_linkedin_results_page(get_page_max=False)
            
            if page_current == page_current_check + 1: 
                # efectivamente pudimos avanzar a la siguiente página
                print('succesfully loaded next results page')
                print('New page: {}'.format(page_current))
                break 
            
        if next_try == 10: 
            print('Exited search due to {} failures to click next results page'.format(next_try))
            sys.exit()
    

# Process

In [193]:
# Open Chromedriver

undetectable_options()

chromedriver_file_loc = '/Users/matias/Documents/Python/chromedriver'
os.chmod(chromedriver_file_loc, 755)
driver = webdriver.Chrome(chromedriver_file_loc, options=options)

actions = ActionChains(driver)


/var/folders/jb/jvyl9b1x6rg1lxy4yhxhb8pw0000gn/T/ipykernel_26026/293295533.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver_file_loc, options=options)


In [194]:
log_in_to_linkedin()

# letsmattion@gmail.com
# LetsGetUsSomeSales88


In [225]:
get_gsheets_inputs()

In [ ]:
### Process 4: Send random DMs out there

## Start

for i in range(len(start_ups_people_df)):
    
    driver.get('https://www.linkedin.com/messaging/thread/new/')
    
    time.sleep(5)
    
    # Type name
    
    driver.find_element(By.XPATH,'//input[@placeholder="Type a name or multiple names"]').send_keys(start_ups_people_df['Name'][2])
    
    time.sleep(5)
    
    # Choose person checking their position
    
    soup = BeautifulSoup(driver.page_source,'html.parser')

    results = soup.find_all("li", {"role": "presentation"})
    
    for j in range(len(results)):
        
        name = results[j].find('strong').get_text()
        
        position = results[j].find('dd').get_text()
        
        if name != start_ups_people_df['Name'][i]: continue
        
        elif start_ups_people_df['Position'][i] not in position: continue
        
        # encontrado, es el nuestro
        
        #driver.find_element(By.XPATH,'/html/body/div[6]/div[3]/div[2]/div/div/main/div/section[2]/div[2]/div[2]/div/div/div/div/ul/li['str(j+1)']/div/button').click()
        driver.find_element(By.XPATH,"//*[contains(text(), '"+position+"')]").click()

        time.sleep(4)
        

        
    
    # Check if we can send DM

    soup = BeautifulSoup(driver.page_source,'html.parser')

    if "With Premium InMail, you can message anyone even if you're not connected. It's 2.6x more effective in hearing back than email or cold calls." in str(soup):
        
        # we cannot DM them
        
        start_ups_people_df.loc[i,'Comment'] = 'Cannot DM'
        
        continue
        
    # We can DM!
    
    # Check if we already have sent DM:
    
    soup = BeautifulSoup(driver.page_source,'html.parser')

    if 'Mattion Tech' in str(soup.find("li", {"class": "msg-s-message-list__event"})):
        
        # already sent some message
        
        # find data in dm table
        
        ### MISSING CODE
        
        # add that we already contacted them
        
        ### MISSING CODE
        
        continue


    ## Construct DM
    
    
    
    
    ## Send DM
    
    

        










In [ ]:
### Process 3.3: Scrape Startups > DM 1 employee every 7 days


# pausado porque me encontré con el search limit




# Employee chosen at random, from all available to DM

## Armarmos lista de empresas ya contactadas en los últimos 7 días


        ### MISSING CODE



## Start

for i in range(len(start_ups_people_df)):
    
    if start_ups_people_df['Company'][i] in dmed_in_last_7d_list: continue
    
    driver.get('https://www.linkedin.com/messaging/thread/new/')
    
    time.sleep(5)
    
    # Type name
    
    driver.find_element(By.XPATH,'//input[@placeholder="Type a name or multiple names"]').send_keys(start_ups_people_df['Name'][2])
    
    time.sleep(5)
    
    # Choose person checking their position
    
    soup = BeautifulSoup(driver.page_source,'html.parser')

    results = soup.find_all("li", {"role": "presentation"})
    
    for j in range(len(results)):
        
        name = results[j].find('strong').get_text()
        
        position = results[j].find('dd').get_text()
        
        if name != start_ups_people_df['Name'][i]: continue
        
        elif start_ups_people_df['Position'][i] not in position: continue
        
        # encontrado, es el nuestro
        
        #driver.find_element(By.XPATH,'/html/body/div[6]/div[3]/div[2]/div/div/main/div/section[2]/div[2]/div[2]/div/div/div/div/ul/li['str(j+1)']/div/button').click()
        driver.find_element(By.XPATH,"//*[contains(text(), '"+position+"')]").click()

        time.sleep(4)
        

        
    
    # Check if we can send DM

    soup = BeautifulSoup(driver.page_source,'html.parser')

    if "With Premium InMail, you can message anyone even if you're not connected. It's 2.6x more effective in hearing back than email or cold calls." in str(soup):
        
        # we cannot DM them
        
        start_ups_people_df.loc[i,'Comment'] = 'Cannot DM'
        
        continue
        
    # We can DM!
    
    # Check if we already have sent DM:
    
    soup = BeautifulSoup(driver.page_source,'html.parser')

    if 'Mattion Tech' in str(soup.find("li", {"class": "msg-s-message-list__event"})):
        
        # already sent some message
        
        # find data in dm table
        
        ### MISSING CODE
        
        # add that we already contacted them
        
        ### MISSING CODE
        
        continue


    ## Construct DM
    
    
    
    
    ## Send DM
    
    

        





In [275]:
soup = BeautifulSoup(driver.page_source,'html.parser')

'Mattion Tech' in str(soup.find("li", {"class": "msg-s-message-list__event"}))




False

In [266]:
for j in range(len(results)):
        
    print(results[j].find('strong').get_text())

Juan Galvez
Juan Espinosa
Juan Antonio Roldán García
Juan P Saavedra Amaya
Juan D.
Juan Bianchi
Juan C. Colon Casanova
Juan Luis Ordóñez Martínez
Juan Ramirez
Juan Masa Santiago


In [268]:
driver.find_element(By.XPATH,"//*[contains(text(), 'Platform Fixed Operations Director')]").click()


In [259]:
driver.find_element(By.XPATH,'/html/body/div[6]/div[3]/div[2]/div/div/main/div/section[2]/div[2]/div[2]/div/div/div/div/ul/li[1]/div/button').click()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[3]/div[2]/div/div/main/div/section[2]/div[2]/div[2]/div/div/div/div/ul/li[1]/div/button"}
  (Session info: chrome=100.0.4896.88)
Stacktrace:
0   chromedriver                        0x000000010052b3d9 chromedriver + 5104601
1   chromedriver                        0x00000001004bbbf3 chromedriver + 4647923
2   chromedriver                        0x00000001000acb48 chromedriver + 392008
3   chromedriver                        0x00000001000e1e26 chromedriver + 609830
4   chromedriver                        0x00000001000e1fe1 chromedriver + 610273
5   chromedriver                        0x0000000100114374 chromedriver + 815988
6   chromedriver                        0x00000001000ff6ed chromedriver + 730861
7   chromedriver                        0x0000000100112091 chromedriver + 807057
8   chromedriver                        0x00000001000ff5b3 chromedriver + 730547


In [ ]:
# click button 

/html/body/div[6]/div[3]/div[2]/div/div/main/div/section[2]/div[2]/div[2]/div/div/div/div/ul/li[1]/div/button

/html/body/div[6]/div[3]/div[2]/div/div/main/div/section[2]/div[2]/div[2]/div/div/div/div/ul/li[2]/div/button
/html/body/div[6]/div[3]/div[2]/div/div/main/div/section[2]/div[2]/div[2]/div/div/div/div/ul/li[6]/div/button

In [248]:
driver.get('https://www.linkedin.com/messaging/thread/new/')

In [262]:
driver.find_element(By.XPATH,'//input[@placeholder="Type a name or multiple names"]').send_keys('juan')


In [250]:
soup = BeautifulSoup(driver.page_source,'html.parser')

results = soup.find_all("li", {"role": "presentation"})

results

[<li role="presentation">
 <div class="entry-point">
 <div></div>
 <button aria-selected="true" class="display-flex flex-row align-items-center msg-connections-typeahead__search-result msg-connections-typeahead__search-result--highlighted" id="ember159" role="option" tabindex="-1" type="button">
 <div class="presence-entity presence-entity--size-2 msg-connections-typeahead__presence-entity">
 <img alt="Photo of Arnav G." class="presence-entity__image EntityPhoto-circle-2 lazy-image ghost-person ember-view" id="ember169" loading="lazy" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7"/>
 <div class="presence-entity__indicator msg-connections-typeahead__presence-entity-indicator presence-entity__indicator--size-2 presence-indicator hidden presence-indicator--size-2">
 <span class="visually-hidden">
       Status is offline
       </span>
 </div>
 </div>
 <div class="msg-connections-typeahead__entity-description">
 <dl class="display-flex flex-column tru

In [251]:
results[0].find('strong').get_text()

'Arnav G.'

In [252]:
results[0].find('dd').get_text()

'Zonal HR Lead East  at Flipkart## Connecting Brains with opportunity'

In [106]:
### Process 3.1: Scrape Startups

for i in range(len(start_ups_searches_df)):

    page_n = 1

    driver.get(start_ups_searches_df['Url'][i])

    time.sleep(4)


    # Scrape current page number

    get_linkedin_results_page(get_page_max=True)


    # loop through all results pages

    while True:

        # Print progress
        
        clear_output(wait=True)
        print('Page {} of {}'.format(page_current,page_max))
        print('{}%'.format(round((page_current-1)*100.0/page_max,1)))
        
        # loop to check we got 10 results (there are always 10 results, maybe unless you are in the final results page)

        results = []

        while (len(results) != 10) or ( (len(results) == 0) & (page_current == page_max) ):

            scroll()

            time.sleep(1)

            soup = BeautifulSoup(driver.page_source,'html.parser')

            results = soup.find_all("li", {"class": "reusable-search__result-container"})
        

        for j in range(len(results)):

            name = results[j].find("span", {"class": "entity-result__title-line"}).get_text().replace('\n','').lstrip(' ').rstrip(' ')

            link = results[j].find("span", {"class": "entity-result__title-line"}).find('a')['href']

            description = results[j].find("div", {"class": "entity-result__primary-subtitle"}).get_text().replace('\n','').lstrip(' ').rstrip(' ')

            followers = results[j].find("div", {"class": "entity-result__secondary-subtitle"}).get_text().replace('\n','').lstrip(' ').rstrip(' ').replace(' followers','')

            try: jobs = results[j].find("div", {"class": "entity-result__simple-insight-text-container"}).get_text().replace('\n','').lstrip(' ').rstrip(' ')
            except: jobs = ''

            location = start_ups_searches_df['Location'][i]

            size = start_ups_searches_df['Size'][i]
            
            search = start_ups_searches_df['Search'][i]

            start_ups_companies_df.loc[len(start_ups_companies_df)] = [name,location,size,followers,jobs,description,search,link]


        # Pagination

        if page_current == page_max: 

            print('finished scraping all start ups')

            break

        else:

            driver.find_element(By.XPATH,'//button[@aria-label="Next"]').click()

            time.sleep(4)

        # check we are in next page
        
        page_current_check = page_current

        get_linkedin_results_page(get_page_max=False)

        if page_current_check != page_current - 1: 

            print('Error: results page number not as expected')
            print('page_current = {}'.format(page_current))
            print('page_current_check = {}'.format(page_current_check+1))

            sys.exit()

    # Paste results in gSheets
    
    start_ups_companies_tab.set_dataframe(start_ups_companies_df, 'A1', fit=False)
    
    print('')
    print('Saved data onto gSheets')
            

Page 100 of 100
100.0%


KeyboardInterrupt: 

In [277]:
### Process 3.2: Scrape Startups > Scrape their employees

# not pefect yet, pero me encontré con el límite de max searches
# linkedin premium ofrece unlimited searches

for i in range(len(start_ups_companies_df)):

    
    ## Filter companies we have already scraped their employees
    
    scraped_companies = list(dict.fromkeys(start_ups_people_df['Company'].tolist()))
    
    if start_ups_companies_df['Name'][i] in scraped_companies: continue
    
    
    print(i)
    print('Scraping employees from company: {}'.format(start_ups_companies_df['Name'][i]))
    
    
    # Open people search

    driver.get('https://www.linkedin.com/search/results/people')

    time.sleep(5)


    ## Add current company filter

    # Open current company filter
    
    driver.find_element(By.XPATH,'//button[@aria-label="Current company filter. Clicking this button displays all Current company filter options."]').click()
    
    time.sleep(2)

    # Enter company name
    
    driver.find_element(By.XPATH,'//input[@placeholder="Add a company"]').send_keys(start_ups_companies_df['Name'][i])
    
    time.sleep(5)
    
    # Keyboard DOWN
    
    actions.send_keys(Keys.DOWN)
    actions.perform()
    
    time.sleep(2)
    
    # Keyboard ENTER
    
    actions.send_keys(Keys.ENTER)
    actions.perform()

    time.sleep(2)

    # Click Show Results button

    try:
        driver.find_element(By.XPATH,'/html/body/div[5]/div[3]/div[2]/section/div/nav/div/ul/li[5]/div/div/div/div[1]/div/form/fieldset/div[2]/button[2]').click()
    except:
        driver.find_element(By.XPATH,'/html/body/div[6]/div[3]/div[2]/section/div/nav/div/ul/li[5]/div/div/div/div[1]/div/form/fieldset/div[2]/button[2]').click()
    
    time.sleep(5)
    
    
    ## Now we scrape
    
    scrape_people()
    
    # edit people scraped to match what we are looking for
    
    people_results_df['company'] = start_ups_companies_df['Name'][i]
    
    # add new results to existing df
    
    for j in range(len(people_results_df)):
        
        start_ups_people_df.loc[len(start_ups_people_df)] = [people_results_df['name'][j]
                                                            ,people_results_df['position'][j]
                                                            ,people_results_df['company'][j]
                                                            ,'No'
                                                            ,people_results_df['link'][j]
                                                            ,''
                                                            ,people_results_df['action'][j]
                                                            ,people_results_df['timestamp'][j]
                                                            ]
    
    # Paste results in gSheets

    start_ups_people_tab.set_dataframe(start_ups_people_df, 'A1', fit=False)

    print('')
    print('Saved data onto gSheets')

    
    
    



Page 7 of 12
50.0%
Error: cannot check that results are correct


SystemExit: 

/Users/matias/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
### Process 2.1: DM w/o Connect > Scrape the connections to DM

### DM without Connect Loop (hacking el 100 Connect request per week limit)

# set run number

run = get_run_number(runs_wo_connect_df)

# global result number

global_n = 0

for i in range(len(searches_df)):
    
    driver.get(searches_df['Link'][i])
    
    time.sleep(5)
    
    print('Search n '+str(i))
    print('')
    
    page_n = 1
    
    # primero scrapeamos todos los contactos de este search, y en un segundo paso les mandamos los DMs
        
    while True:
        
        # para cada página de resultados

        soup = BeautifulSoup(driver.page_source,'html.parser')
        
        results = soup.find_all("li", {"class": "reusable-search__result-container"})
        
        print('')
        print('Results page n: {}'.format(page_n))
        
        for j in range(len(results)):
            
            global_n += 1
            
            #print('')
            #print('Profile {}'.format(global_n))
        
            # get details of every results
            
            
            # name

            name = results[j].find('span',{'class':'entity-result__title-text'}).find("a", {"class": "app-aware-link"}).get_text().replace('\n','')

            if 'View' in name: name = name[:name.find('View')]
            
            #print('Name: {}'.format(name))

            
            # position

            position = results[j].find('div',{'class':'entity-result__primary-subtitle'}).get_text().replace('\n','')
            
            #print('Position: {}'.format(position))

            
            # link

            if name == 'LinkedIn Member': link = ''

            else:

                link_full = results[j].find('span',{'class':'entity-result__title-text'}).find("a", {"class": "app-aware-link"})['href']
                
                link = link_full[:link_full.find('?')]

                
            # company

            try: 
                sub_position_full = results[j].find('p',{'class':'entity-result__summary'}).get_text().replace('\n','')
            except:
                sub_position_full = 'empty'

            if 'Current: ' not in sub_position_full: sub_position = ''

            else: sub_position = sub_position_full[9:]

            if   ' at ' in position: company = position[position.find(' at ')+4:]
            elif ' @ '  in position: company = position[position.find(' @ ')+4:]
            
            elif ' at ' in sub_position: company = sub_position[sub_position.find(' at ')+4:]
            elif ' @ '  in sub_position: company = sub_position[sub_position.find(' @ ')+4:]

            else: company = ''
            
            #print('Company: {}'.format(company))
            
            
            # action available
            
            try: 
                #action = results[j].find('button')['aria-label']
                action = results[j].find("div", {"class": "entity-result__actions"}).find("span", {"class": "artdeco-button__text"}).get_text().lstrip('\n').rstrip('\n').lstrip(' ').rstrip(' ')
            except:
                action = 'None'
                
            #print('Action available: {}'.format(action))
            
            
            ## comment
            
            comment = ''
            
            # exclude people looking for a new opportunity
            
            if 'new opport' in position.lower(): comment = 'Looking for new opportunities'
                
            
            ### Incorporate data
            
            # First lets check if we already have this person in our database
            
            try:
                
                run_row = runs_wo_connect_df.loc[runs_wo_connect_df['Link'] == link].index.to_list()[0]
                
                # encontrado
                
                # update data
                
                runs_wo_connect_df.loc[run_row,'Name'] = name
                runs_wo_connect_df.loc[run_row,'Position'] = position
                runs_wo_connect_df.loc[run_row,'Company'] = company
                runs_wo_connect_df.loc[run_row,'Action'] = action
                
                if 'new opport' in position.lower(): runs_wo_connect_df.loc[run_row,'Comment'] = comment
                
            except:
                
                # no se encuentra en nuestra base de datos
                
                runs_wo_connect_df.loc[len(runs_wo_connect_df)] = [name,position,company,False,link,comment,action,run,page_n,global_n,now_str()]
            
            #dms_wo_connect_df.loc[len(dms_wo_connect_df)] = [name,position,company,False,link,'',custom_dm,now_str()]
            
            #print('Saved data onto dfs')
        
        
        
        
        # input to stop and analyze
        #order = input("ingresar ok para siguiente página o stop para parar")
        #if order == 'stop':
        #    break
        
        # click next page of results
        
        scroll()
        time.sleep(1)
        print('')
        next_try = 0
        
        #if page_n == 3: 
        #    print('intentional stop after 3 pages to check results')
        #    break 
            
            
            
        # Open next results page
        
        if page_n == 100: break # linkedin give back max 100 page results
        
        next_try = 0
        
        while next_try < 10:
            
            failed_next_try = 0
            
            try:
                #driver.find_element(By.XPATH,"//*[contains(text(), 'Next')]")#.click()
                driver.find_element(By.XPATH,'//*[@aria-label="Next"]').click()
                time.sleep(6)
                print('Clicked next results page')
                next_try += 1

            except:
                print('Failed to click next resulst page')
                time.sleep(1)
                failed_next_try = 1
                continue
        
            try:
                # click No Results found button
                driver.find_element(By.XPATH,'//*[@data-test="no-results-cta"]').click()
                time.sleep(5)
                print('Detected results LinkedIn error: Clicked re-load results button.')

            except:
                pass
            
            if failed_next_try == 1: continue
            
            # check if new page is ok
            
            new_page_n = get_page_number()
            
            if new_page_n == page_n + 1: 
                # efectivamente pudimos avanzar a la siguiente página
                print('succesfully loaded next results page')
                page_n = new_page_n
                print('New page: {}'.format(page_n))
                break 
            
        if next_try == 10: 
            print('Exited search due to {} failures to click next results page'.format(next_try))
            break
            
    # Paste data in gSheets
    
    dms_wo_connect_tab.set_dataframe(dms_wo_connect_df, 'A1', fit=False)
    runs_wo_connect_tab.set_dataframe(runs_wo_connect_df, 'A1', fit=False)
    
    print('')
    print('Saved data onto gSheets')

            
    ## Ahora a mandar todos los DMs
    
    for k in range(len(runs_wo_connect_df)):
        
        
        # Get usable data
        
        name = runs_wo_connect_df['Name'][k]
        position = runs_wo_connect_df['Position'][k]
        company = runs_wo_connect_df['Company'][k]
        link = runs_wo_connect_df['Link'][k]
        comment = runs_wo_connect_df['Comment'][k]
        
        
        # Already contacted exclusion
        
        if runs_wo_connect_df['Contacted?'][k] == 'TRUE': continue
        
        # Cannot send dm exclusion
        
        if runs_wo_connect_df['Action'][k] != 'Connect': continue
        
        # Searching for new opportunities exlcusion
        
        if comment == 'Looking for new opportunities': continue
        
        # DM action locked exclusion
        
        if comment == 'DM action locked': continue
        
        # Open to work exclusion
        
        if comment == 'Open to work': continue
        
        
        # Open profile
        
        driver.get(runs_wo_connect_df['Link'][k])
        
        time.sleep(5)
        
        # Are they Open to Work?
        
        soup = BeautifulSoup(driver.page_source,'html.parser')

        if '#OPEN_TO_WORK' in str(soup):
            
            runs_wo_connect_df.loc[k,'Comment'] = 'Open to work'
            #runs_wo_connect_df['Comment'][k] = 'Open to work'
            
            continue
        
        
        # Can we dm them?
        
        soup = BeautifulSoup(driver.page_source,'html.parser')

        #action_buttons = soup.find("div", {"class": "pvs-profile-actions"}).find_all('button')
        #
        #can_dm = True # porque si no tiene candado el button, no aparece como 'button' en el htmml
        #
        #for action_button in action_buttons:
        #
        #    button_text = action_button.get_text().lstrip('\n').rstrip('\n').lstrip(' ').rstrip(' ').lstrip('\n').rstrip('\n').lstrip(' ').rstrip(' ')
        #
        #    if button_text != 'Message': continue
        #
        #    if 'lock' in str(action_button).lower():
        #
        #        # cannot message, has lock icon on it
        #
        #        runs_wo_connect_df['Comment'][k] = 'DM action locked'
        #
        #        can_dm = False
        #
        #    else:
        #
        #        can_dm = True
        #
        #    break
        #    
        #if can_dm == False: continue

        try:
            
            dm_link = 'https://www.linkedin.com' + soup.find("a", {"class": "message-anywhere-button"})['href']

        except:
            
            # cannot dm
            #runs_wo_connect_df['Comment'][k] = 'DM action locked'
            runs_wo_connect_df.loc[k,'Comment'] = 'DM action locked'
            continue
            
            
        # Verification Loop
        # infinite loop warning
        
        while True:
            
            # Open DM Link

            driver.get(dm_link)

            time.sleep(5)

            # Confirm that chat opened is with correct contact

            soup = BeautifulSoup(driver.page_source,'html.parser')

            dm_recipient_html = soup.find("a", {"class": "msg-compose__profile-link ember-view"})
            
            if dm_recipient_html == None: dm_recipient_html = soup.find("a", {"class": "profile-card-one-to-one__profile-link ember-view"})
            # en este caso el soup.find anterior no trajo nada, es porque el chat abierto ya tiene mensaje
            # pero hay que verificar que este chat es efectivamente de la persona que le queremos mandar dm
            # linkedin podría abrir un chat de otra persona

            # test 1
            
            if dm_recipient_html['href'].rstrip('/') != runs_wo_connect_df['Link'][k].replace('https://www.linkedin.com',''): continue    # "/in/michaellwalter/": continue

            # test 2
            
            dm_recipient_name = dm_recipient_html.get_text().lstrip('\n').rstrip('\n').lstrip(' ').rstrip(' ').lstrip('\n').rstrip('\n').lstrip(' ').rstrip(' ')
            
            if runs_wo_connect_df['Name'][k] == dm_recipient_name: break # all is ok and we exit verification loop

            #dm_container = soup.find("div", {"class": "msg-inmail-compose-form-v2"})
            #'Mike Walter' in str(dm_container)
            #"/in/michaellwalter/" in str(dm_container)

        
        # customize DM

        dm_name = name.split(' ')[0]

        dm_position = position

        if '|' in dm_position: dm_position = dm_position[:position.find('|')]
        if ',' in dm_position: dm_position = dm_position[:dm_position.find(',')]
        if '&' in dm_position: dm_position = dm_position[:dm_position.find('&')]
        if '/' in dm_position: dm_position = dm_position[:dm_position.find('/')]
        if '-' in dm_position: dm_position = dm_position[:dm_position.find('-')]

        dm_position = dm_position.replace('Operations','Ops')
        dm_position = dm_position.replace('Senior','Sr.')
        dm_position = dm_position.replace('Vice President','VP')
        dm_position = dm_position.replace('Of','of')
        dm_position = dm_position.replace('Business Development','BD')

        if ' at ' in dm_position: dm_position = dm_position[:dm_position.find(' at ')]

        elif ' @ ' in dm_position: dm_position = dm_position[:dm_position.find(' @ ')]

        dm_position = dm_position.lstrip(' ').rstrip(' ')

        #print('DM Position: {}'.format(dm_position))

        #dm_message_direct_1_full_title

        custom_dm_body = dm_message_direct_1_full_body.replace('<<name>>',dm_name).replace('<<position>>',dm_position)

        
        # Fill dm title
        
        try:
            title_input = driver.find_element(By.XPATH,'//input[@class=" artdeco-text-input--input"][@name="subject"]')
            time.sleep(0.5)
            title_input.send_keys(dm_message_direct_1_full_title)
            time.sleep(0.5)
            
        except:
            # if we fail to select input it is probably since we have already sent a dm to this person
            # lets verify this, since I wouldn't want to assume this and then assume I sent a DM to someone just because the previous step failed
            
            soup = BeautifulSoup(driver.page_source,'html.parser')
            
            sent_dm_html = soup.find("div", {"class": "msg-s-event-listitem"})
            
            if 'Mattion Tech' in str(sent_dm_html):
                # already sent dm
                runs_wo_connect_df.loc[k,'Contacted?'] = True
                dms_wo_connect_df.loc[len(dms_wo_connect_df)] = [name,position,company,True,link,'Error: already sent dm','Error: already sent dm',now_str()]
                continue
        
        # Fill dm body
        
        dm_input = driver.find_element(By.XPATH,'//div[@role="textbox"][@aria-label="Write a message…"]')  #[@class="msg-form__contenteditable"]
        time.sleep(0.5)
        dm_input.send_keys(custom_dm_body)
        time.sleep(0.5)
        
        
        # Click Send button
        
        driver.find_element(By.XPATH,'//button[text()="Send"]').click()  # [@class="msg-form__send-button"] [type()="submit"]
        # dm sent
        time.sleep(5)
        
        # Save DM data
    
        dms_wo_connect_df.loc[len(dms_wo_connect_df)] = [name,position,company,True,link,comment,custom_dm_body,now_str()]

        runs_wo_connect_df.loc[k,'Contacted?'] = True
        
        #runs_wo_connect_df.loc[len(runs_wo_connect_df)] = [name,position,company,False,link,comment,action,run,page_n,global_n,now_str()]
        
        
    # Paste data in gSheets

    dms_wo_connect_tab.set_dataframe(dms_wo_connect_df, 'A1', fit=False)
    runs_wo_connect_tab.set_dataframe(runs_wo_connect_df, 'A1', fit=False)

    print('')
    print('Saved data onto gSheets')

    
    

In [ ]:
### Process 2.2: DM w/o Connect > Send the DMs

## Ahora a mandar todos los DMs

for k in range(len(runs_wo_connect_df)):


    # Get usable data

    name = runs_wo_connect_df['Name'][k]
    position = runs_wo_connect_df['Position'][k]
    company = runs_wo_connect_df['Company'][k]
    link = runs_wo_connect_df['Link'][k]
    comment = runs_wo_connect_df['Comment'][k]


    # Already contacted exclusion

    if runs_wo_connect_df['Contacted?'][k] == 'TRUE': continue

    # Cannot send dm exclusion

    if runs_wo_connect_df['Action'][k] != 'Connect': continue

    # Searching for new opportunities exlcusion

    if comment == 'Looking for new opportunities': continue

    # DM action locked exclusion

    if comment == 'DM action locked': continue

    # Open to work exclusion

    if comment == 'Open to work': continue


    # Open profile

    driver.get(runs_wo_connect_df['Link'][k])

    time.sleep(5)

    # Are they Open to Work?

    soup = BeautifulSoup(driver.page_source,'html.parser')

    if '#OPEN_TO_WORK' in str(soup):

        runs_wo_connect_df.loc[k,'Comment'] = 'Open to work'
        #runs_wo_connect_df['Comment'][k] = 'Open to work'

        continue


    # Can we dm them?

    soup = BeautifulSoup(driver.page_source,'html.parser')

    #action_buttons = soup.find("div", {"class": "pvs-profile-actions"}).find_all('button')
    #
    #can_dm = True # porque si no tiene candado el button, no aparece como 'button' en el htmml
    #
    #for action_button in action_buttons:
    #
    #    button_text = action_button.get_text().lstrip('\n').rstrip('\n').lstrip(' ').rstrip(' ').lstrip('\n').rstrip('\n').lstrip(' ').rstrip(' ')
    #
    #    if button_text != 'Message': continue
    #
    #    if 'lock' in str(action_button).lower():
    #
    #        # cannot message, has lock icon on it
    #
    #        runs_wo_connect_df['Comment'][k] = 'DM action locked'
    #
    #        can_dm = False
    #
    #    else:
    #
    #        can_dm = True
    #
    #    break
    #    
    #if can_dm == False: continue

    try:

        dm_link = 'https://www.linkedin.com' + soup.find("a", {"class": "message-anywhere-button"})['href']

    except:

        # cannot dm
        #runs_wo_connect_df['Comment'][k] = 'DM action locked'
        runs_wo_connect_df.loc[k,'Comment'] = 'DM action locked'
        continue


    # Verification Loop
    # infinite loop warning

    while True:

        # Open DM Link

        driver.get(dm_link)

        time.sleep(5)

        # Confirm that chat opened is with correct contact

        soup = BeautifulSoup(driver.page_source,'html.parser')

        dm_recipient_html = soup.find("a", {"class": "msg-compose__profile-link ember-view"})

        if dm_recipient_html == None: dm_recipient_html = soup.find("a", {"class": "profile-card-one-to-one__profile-link ember-view"})
        # en este caso el soup.find anterior no trajo nada, es porque el chat abierto ya tiene mensaje
        # pero hay que verificar que este chat es efectivamente de la persona que le queremos mandar dm
        # linkedin podría abrir un chat de otra persona

        # test 1

        if dm_recipient_html['href'].rstrip('/') != runs_wo_connect_df['Link'][k].replace('https://www.linkedin.com',''): continue    # "/in/michaellwalter/": continue

        # test 2

        dm_recipient_name = dm_recipient_html.get_text().lstrip('\n').rstrip('\n').lstrip(' ').rstrip(' ').lstrip('\n').rstrip('\n').lstrip(' ').rstrip(' ')

        if runs_wo_connect_df['Name'][k] == dm_recipient_name: break # all is ok and we exit verification loop

        #dm_container = soup.find("div", {"class": "msg-inmail-compose-form-v2"})
        #'Mike Walter' in str(dm_container)
        #"/in/michaellwalter/" in str(dm_container)


    # customize DM

    dm_name = name.split(' ')[0]

    dm_position = position

    if '|' in dm_position: dm_position = dm_position[:position.find('|')]
    if ',' in dm_position: dm_position = dm_position[:dm_position.find(',')]
    if '&' in dm_position: dm_position = dm_position[:dm_position.find('&')]
    if '/' in dm_position: dm_position = dm_position[:dm_position.find('/')]
    if '-' in dm_position: dm_position = dm_position[:dm_position.find('-')]

    dm_position = dm_position.replace('Operations','Ops')
    dm_position = dm_position.replace('Senior','Sr.')
    dm_position = dm_position.replace('Vice President','VP')
    dm_position = dm_position.replace('Of','of')
    dm_position = dm_position.replace('Business Development','BD')

    if ' at ' in dm_position: dm_position = dm_position[:dm_position.find(' at ')]

    elif ' @ ' in dm_position: dm_position = dm_position[:dm_position.find(' @ ')]

    dm_position = dm_position.lstrip(' ').rstrip(' ')

    #print('DM Position: {}'.format(dm_position))

    #dm_message_direct_1_full_title

    custom_dm_body = dm_message_direct_1_full_body.replace('<<name>>',dm_name).replace('<<position>>',dm_position)


    # Fill dm title

    try:
        title_input = driver.find_element(By.XPATH,'//input[@class=" artdeco-text-input--input"][@name="subject"]')
        time.sleep(0.5)
        title_input.send_keys(dm_message_direct_1_full_title)
        time.sleep(0.5)

    except:
        # if we fail to select input it is probably since we have already sent a dm to this person
        # lets verify this, since I wouldn't want to assume this and then assume I sent a DM to someone just because the previous step failed

        soup = BeautifulSoup(driver.page_source,'html.parser')

        sent_dm_html = soup.find("div", {"class": "msg-s-event-listitem"})

        if 'Mattion Tech' in str(sent_dm_html):
            # already sent dm
            runs_wo_connect_df.loc[k,'Contacted?'] = True
            dms_wo_connect_df.loc[len(dms_wo_connect_df)] = [name,position,company,True,link,'Error: already sent dm','Error: already sent dm',now_str()]
            continue

    # Fill dm body

    dm_input = driver.find_element(By.XPATH,'//div[@role="textbox"][@aria-label="Write a message…"]')  #[@class="msg-form__contenteditable"]
    time.sleep(0.5)
    dm_input.send_keys(custom_dm_body)
    time.sleep(0.5)


    # Click Send button

    driver.find_element(By.XPATH,'//button[text()="Send"]').click()  # [@class="msg-form__send-button"] [type()="submit"]
    # dm sent
    time.sleep(5)

    # Save DM data

    dms_wo_connect_df.loc[len(dms_wo_connect_df)] = [name,position,company,True,link,comment,custom_dm_body,now_str()]

    runs_wo_connect_df.loc[k,'Contacted?'] = True

    #runs_wo_connect_df.loc[len(runs_wo_connect_df)] = [name,position,company,False,link,comment,action,run,page_n,global_n,now_str()]


# Paste data in gSheets

dms_wo_connect_tab.set_dataframe(dms_wo_connect_df, 'A1', fit=False)
runs_wo_connect_tab.set_dataframe(runs_wo_connect_df, 'A1', fit=False)

print('')
print('Saved data onto gSheets')




In [ ]:
### Process 1: Connect with DM

### Connect Loop (nos vimos cagados por el max 100 connect requests por semana)

# efectivamente aún si le das al botón de Got it, no se manda el Connect que le diste. funciona bien el límite de linkedin


# set run number

run = get_run_number(runs_df)

# global result number

global_n = 0

for i in range(len(searches_df)):
    
    driver.get(searches_df['Link'][i])
    
    time.sleep(5)
    
    print('Search n '+str(i))
    print('')
    
    page_n = 1
        
    while True:
        
        # para cada página de resultados

        soup = BeautifulSoup(driver.page_source,'html.parser')
        
        results = soup.find_all("li", {"class": "reusable-search__result-container"})
        
        print('')
        print('Results page n: {}'.format(page_n))
        
        for j in range(len(results)):
            
            global_n += 1
            
            print('')
            print('Profile {}'.format(global_n))
        
            # get details of every results
            
            
            # name

            name = results[j].find('span',{'class':'entity-result__title-text'}).find("a", {"class": "app-aware-link"}).get_text().replace('\n','')

            if 'View' in name: name = name[:name.find('View')]
            
            print('Name: {}'.format(name))

            
            # position

            position = results[j].find('div',{'class':'entity-result__primary-subtitle'}).get_text().replace('\n','')
            
            print('Position: {}'.format(position))

            
            # link

            if name == 'LinkedIn Member': link = ''

            else:

                link_full = results[j].find('span',{'class':'entity-result__title-text'}).find("a", {"class": "app-aware-link"})['href']
                
                link = link_full[:link_full.find('?')]

                
            # company

            try: 
                sub_position_full = results[j].find('p',{'class':'entity-result__summary'}).get_text().replace('\n','')
            except:
                sub_position_full = 'empty'

            if 'Current: ' not in sub_position_full: sub_position = ''

            else: sub_position = sub_position_full[9:]

            if   ' at ' in position: company = position[position.find(' at ')+4:]
            elif ' @ '  in position: company = position[position.find(' @ ')+4:]
            
            elif ' at ' in sub_position: company = sub_position[sub_position.find(' at ')+4:]
            elif ' @ '  in sub_position: company = sub_position[sub_position.find(' @ ')+4:]

            else: company = ''
            
            print('Company: {}'.format(company))
            
            
            # customize DM
            
            dm_name = name.split(' ')[0]
            
            
            dm_position = position
            
            if '|' in dm_position: dm_position = dm_position[:position.find('|')]
            if ',' in dm_position: dm_position = dm_position[:dm_position.find(',')]
            if '&' in dm_position: dm_position = dm_position[:dm_position.find('&')]
            if '/' in dm_position: dm_position = dm_position[:dm_position.find('/')]
            if '-' in dm_position: dm_position = dm_position[:dm_position.find('-')]

            dm_position = dm_position.replace('Operations','Ops')
            dm_position = dm_position.replace('Senior','Sr.')
            dm_position = dm_position.replace('Vice President','VP')
            dm_position = dm_position.replace('Of','of')
            dm_position = dm_position.replace('Business Development','BD')

            if ' at ' in dm_position: dm_position = dm_position[:dm_position.find(' at ')]

            elif ' @ ' in dm_position: dm_position = dm_position[:dm_position.find(' @ ')]
            
            dm_position = dm_position.lstrip(' ').rstrip(' ')

            print('DM Position: {}'.format(dm_position))
            

            custom_dm = dm_message.replace('<<name>>',dm_name).replace('<<position>>',dm_position)

            
            # action available
            
            try: 
                #action = results[j].find('button')['aria-label']
                action = results[j].find("div", {"class": "entity-result__actions"}).find("span", {"class": "artdeco-button__text"}).get_text().lstrip('\n').rstrip('\n').lstrip(' ').rstrip(' ')
            except:
                action = 'None'
                
            print('Action available: {}'.format(action))
            
            
            # exclude people looking for a new opportunity
            
            if 'new opport' in position.lower():
                runs_df.loc[len(runs_df)] = [name,position,company,False,link,'Looking for new opportunities',custom_dm,action,run,now_str()]
                print('Looking for new opportunities')
                continue
            
            
            #custom_dm
            
            if len(custom_dm) > 300:
                
                # lets try to reduce the dm_position length
                if 'VP' in dm_position: dm_position = 'VP'
                elif 'Director' in dm_position: dm_position = 'Director'
                elif 'Head' in dm_position: dm_position = 'Head'
                elif 'General Manager' in dm_position: dm_position = 'General Manager'
                elif 'Manager' in dm_position: dm_position = 'Manager'
                else: pass
            
                custom_dm = dm_message.replace('<<name>>',dm_name).replace('<<position>>',dm_position)
                
                if len(custom_dm) > 300:
                
                    #dms_df.loc[len(dms_df)] = [name,position,company,False,link,'DM over 300 char',custom_dm]
                    runs_df.loc[len(runs_df)] = [name,position,company,False,link,'DM over 300 char',custom_dm,action,run,now_str()]
                    print('DM over 300 char')
                    continue
                
            
            # Connect
            
            if action != 'Connect':
                
                #dms_df.loc[len(dms_df)] = [name,position,company,False,link,'Action available: '+action,custom_dm]
                runs_df.loc[len(runs_df)] = [name,position,company,False,link,'Action available: '+action,custom_dm,action,run,now_str()]
                print('No Connect action available. Go to next result.')
                continue
                
                
            #### no perdemos nada mandandole el pitch a una empresa super grosa...
            ## Analyze company exclusions
            #
            #if company in company_exclusions_list:
            #    
            #    #dms_df.loc[len(dms_df)] = [name,position,company,False,link,'Company exclusion',custom_dm]
            #    runs_df.loc[len(runs_df)] = [name,position,company,False,link,'Company exclusion',custom_dm,action,run,now_str()]
            #    print('Company exclusion')
            #    continue
            

            # click "Connect" button

            connect_result = click_connect_button()
            
            if connect_result == 'failed':
                
                #dms_df.loc[len(dms_df)] = [name,position,company,False,link,'Not available to connect',custom_dm]
                runs_df.loc[len(runs_df)] = [name,position,company,False,link,'Not available to connect',custom_dm,action,run,now_str()]
                print('Failed to click Connect button')
                continue

                
            # click "Add a note"

            #driver.find_element(By.XPATH,"//*[contains(text(), 'Add a note')]")#.click()

            driver.find_element(By.XPATH,'//*[@aria-label="Add a note"]').click()
            
            print('Clicked Add a note button')
            
            time.sleep(1)


            # insert DM
    
            driver.find_element(By.XPATH,'//*[@id="custom-message"]').send_keys(custom_dm)
        
            print('Added DM')
            
            time.sleep(1)
            
            
            # click Send button

            #driver.find_element(By.XPATH,"//*[contains(text(), 'Send')]").click()

            driver.find_element(By.XPATH,'//*[@aria-label="Send now"]').click()
            
            print('Sent Connect request with DM')
            
            time.sleep(1)
            
            
            ##### Have we reached the 100 weekly limit? Dont care
            ####
            ####try:
            ####    driver.find_element(By.XPATH,'//*[@aria-label="Got it"]').click()
            ####    print('Clicked weekly connect limit pop-up')
            ####    time.sleep(1)
            ####except:
            ####    pass
            ####    # no pop-up
                
            
            # Save data
            
            dms_df.loc[len(dms_df)] = [name,position,company,True,link,'',custom_dm,now_str()]
            
            runs_df.loc[len(runs_df)] = [name,position,company,True,link,'',custom_dm,action,run,now_str()]
            
            print('Saved data onto dfs')
        
        
        
        
        # input to stop and analyze
        #order = input("ingresar ok para siguiente página o stop para parar")
        #if order == 'stop':
        #    break
        
        # click next page of results
        
        scroll()
        time.sleep(1)
        print('')
        next_try = 0
        
        #if page_n == 3: 
        #    print('intentional stop after 3 pages to check results')
        #    break 
            
            
            
        # Open next results page
        
        next_try = 0
        
        while next_try < 10:
            
            failed_next_try = 0
            
            try:
                #driver.find_element(By.XPATH,"//*[contains(text(), 'Next')]")#.click()
                driver.find_element(By.XPATH,'//*[@aria-label="Next"]').click()
                time.sleep(6)
                print('Clicked next results page')
                next_try += 1

            except:
                print('Failed to click next resulst page')
                time.sleep(1)
                failed_next_try = 1
                continue
        
            try:
                # click No Results found button
                driver.find_element(By.XPATH,'//*[@data-test="no-results-cta"]').click()
                time.sleep(5)
                print('Detected results LinkedIn error: Clicked re-load results button.')

            except:
                pass
            
            if failed_next_try == 1: continue
            
            # check if new page is ok
            
            new_page_n = get_page_number()
            
            if new_page_n == page_n + 1: 
                # efectivamente pudimos avanzar a la siguiente página
                print('succesfully loaded next results page')
                page_n = new_page_n
                print('New page: {}'.format(page_n))
                break 
            
        if next_try == 10: 
            print('Exited search due to {} failures to click next results page'.format(next_try))
            break
            
        
        
# Paste data in gSheets
#dms_tab = gs.open_by_key(sheet_id).worksheet_by_title('DMs')

dms_tab.set_dataframe(dms_df, 'A1', fit=False)
runs_tab.set_dataframe(runs_df, 'A1', fit=False)
print('')
print('Saved data onto gSheets')
